In [0]:
"""
    https://www.youtube.com/watch?v=rGG6FAYHDNc&list=PLBTZqjSKn0IfuIqbMIqzS-waofsPHMS0E&index=26 
    Write a SQL to find busiest route with total ticket count.
    oneway_round = O -> One way trip
    oneway_round = R -> Round trip
    Note DEL to BOM is different than BOM to DEL
"""
from pyspark.sql.functions import *
from pyspark.sql.window import Window

tickets = spark.createDataFrame([
    ('DEF456', 'BOM', 'DEL', 'O', 150),
    ('GHI789', 'DEL', 'BOM', 'R', 50),
    ('JKL012', 'BOM', 'DEL', 'R', 75),
    ('MNO345', 'DEL', 'NYC', 'O', 200),
    ('PQR678', 'NYC', 'DEL', 'O', 180),
    ('STU901', 'NYC', 'DEL', 'R', 60),
    ('ABC123', 'DEL', 'BOM', 'O', 100),
    ('VWX234', 'DEL', 'NYC', 'R', 90)
], ["airline_number", "origin", "destination", "oneway_round", "ticket_count"]
)

tickets.show()
tickets.printSchema()

+--------------+------+-----------+------------+------------+
|airline_number|origin|destination|oneway_round|ticket_count|
+--------------+------+-----------+------------+------------+
|        DEF456|   BOM|        DEL|           O|         150|
|        GHI789|   DEL|        BOM|           R|          50|
|        JKL012|   BOM|        DEL|           R|          75|
|        MNO345|   DEL|        NYC|           O|         200|
|        PQR678|   NYC|        DEL|           O|         180|
|        STU901|   NYC|        DEL|           R|          60|
|        ABC123|   DEL|        BOM|           O|         100|
|        VWX234|   DEL|        NYC|           R|          90|
+--------------+------+-----------+------------+------------+

root
 |-- airline_number: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- oneway_round: string (nullable = true)
 |-- ticket_count: long (nullable = true)



In [0]:
# Since data is shinked, we need to exapad for round trips using union

df1 = tickets.select("origin","destination","oneway_round","ticket_count")

df2 = tickets.filter(col("oneway_round") == "R") \
    .select("destination","origin","oneway_round","ticket_count")

df1.union(df2).show()

+------+-----------+------------+------------+
|origin|destination|oneway_round|ticket_count|
+------+-----------+------------+------------+
|   BOM|        DEL|           O|         150|
|   DEL|        BOM|           R|          50|
|   BOM|        DEL|           R|          75|
|   DEL|        NYC|           O|         200|
|   NYC|        DEL|           O|         180|
|   NYC|        DEL|           R|          60|
|   DEL|        BOM|           O|         100|
|   DEL|        NYC|           R|          90|
|   BOM|        DEL|           R|          50|
|   DEL|        BOM|           R|          75|
|   DEL|        NYC|           R|          60|
|   NYC|        DEL|           R|          90|
+------+-----------+------------+------------+



In [0]:
df1.union(df2) \
    .groupBy("origin","destination").agg(sum("ticket_count").alias("total_")) \
    .orderBy(desc("total_"))\
    .limit(1)\
    .show()

+------+-----------+------+
|origin|destination|total_|
+------+-----------+------+
|   DEL|        NYC|   350|
+------+-----------+------+

